# Amazon SageMaker Data Wrangler for Churn Data Preprocessing

Amazon SageMaker Data Wrangler は、単一のビジュアルインターフェイスからデータ選択、クレンジング、探索、視覚化など、データ準備ワークフローの各ステップを簡素化して実現するツールです。

このサンプルでは、音楽ストリーミングサイトのユーザーアクティビティログから顧客離脱予測に使うための特徴量エンジニアリングを行います。

このサンプルを実行するには、Amazon SageMaker Studio でこのノートブックを開いて実行してください。

## 準備

In [ ]:
import sagemaker
import json
import pandas as pd

In [ ]:
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()  # replace with your own bucket name if you have one
prefix = "churn"

まずは、今回使用するユーザーアクティビティログを読み込み、スキーマを確認します。

このサンプルで使用するデータは [EventSim](https://github.com/Interana/eventsim) で生成された1年間（2019/10/28 - 2020/10/28）の
音楽ストリーミングサイトのユーザーアクティビティーデータのデータセットです。

ユーザーのアクティビティごとにレコードがある状態です。これを集計してユーザーごとのレコードに変換していきます。

このデータを Python で分析するサンプルについては [こちらのページ](https://sagemaker-examples.readthedocs.io/en/latest/use-cases/customer_churn/1_cust_churn_dataprep.html#Feature-Engineering-with-SageMaker-Processing) をご参照ください。

In [ ]:
!aws s3 cp s3://sagemaker-sample-files/datasets/tabular/customer-churn/customer-churn-data-v2.zip ./data/raw/customer-churn-data.zip

In [ ]:
!unzip -p ./data/raw/customer-churn-data.zip sample.csv > ./data/sample_data.csv

In [ ]:
data = pd.read_csv("data/sample_data.csv")
data.head(5)

スキーマを確認できたら、S3 にアップロードします

In [ ]:
input_data_path = sagemaker_session.upload_data('data/sample_data.csv', bucket=bucket, key_prefix= f"{prefix}/input")
input_data_path

次に、アップロードした S3 の URI を Amazon SageMaker Data Wrangler で使用する `dw_example.flow` に書き込みます。

In [ ]:
flow_file = "dw_example.flow"

# read flow file and change the s3 location to our `processing_output_filename`
with open(flow_file, "r") as f:
    flow = f.read()

    flow = json.loads(flow)
    flow["nodes"][0]["parameters"]["dataset_definition"]["s3ExecutionContext"][
        "s3Uri"
    ] = input_data_path

with open("dw_example.flow", "w") as f:
    json.dump(flow, f)

これで準備が整いましたので、`dw_example.flow` をクリックして SageMaker Studio で SageMaker Data Wrangler を開きます。

## 特徴量エンジニアリングワークフローの実行

以降のステップは SageMaker Data Wrangler のタブで行います。

SageMaker Data Wrangler を開くと最初にワークフローが表示されます。

S3 からデータをインポートし、データ型を指定し、複数の変換 Step を実行しています。

![](images/flow.png)

Steps にマウスをホバーすると **+** マークが出現するのでクリックして **Add Transform** を選択します。

![](images/add_transform.png)

そうすると、データとデータに与えられた変換の一覧が表示されます。

ステップを選択すると、そのステップの変換の内容とそのステップを実行したあとのデータが表示されます。

どのような変換が使用されているのか確認してみましょう。

複数のステップがありますが大まかに以下のようなステップになっています

1. csv データを s3 から取得
2. データの各列の型を指定
3. 必要のない行と列を削除する
4. 学習に使うデータ範囲を決め、顧客離脱（churn）を定義し列を追加。
    - 2020/07/01 以前のデータを学習に使い、それ以降で `Cancellation Confirmation` イベントがあれば顧客離脱とする
5. sessionId や userId をもとにグループして集計して列を追加
    - sessionId でグルーピングし、セッションごとのアクティビティや活動時間を集計する
    - userId でグルーピングし、様々な特徴を作成する
6. 必要のない列を削除し、重複排除を行うことでユーザーごとの行にする

![](images/transform_list.png)

データの前処理が完了したら、Data flow に戻り、Steps の **+** マークをクリックして Add Destination から s3 に csv としてエクスポートする設定をします。

![](images/export_s3.png)

画面右上の **Create job** からジョブを作成し実行すると SageMaker Processing によるジョブが実行され s3 に結果が出力されます。

![](images/create_job.png)

処理が終わったら Notebook と Data Wrangler で使用しているインスタンスを終了します。
明示的にインスタンスを終了しないと起動されたままなので気をつけてください。

![](images/terminate.png)

## Next Step

このサンプルでは Amazon SageMaker Data Wrangler を使用してデータの特徴量エンジニアリングを行いました。

出力されたデータからそのまま SageMaker Autopilot でモデルを学習させることも可能ですし、SageMaker Canvas を使用してインタラクティブにモデルを作成することも可能です。

今回は試行錯誤のためにユーザー数が 100 人ほどの小さなデータセットの `sample_data.csv` を使用しましたが、モデルを学習させる際は最低 500 人ほどのデータが必要になります。

次のステップとして自社のデータか、[こちら](https://sagemaker-examples.readthedocs.io/en/latest/use-cases/customer_churn/1_cust_churn_dataprep.html#Feature-Engineering-with-SageMaker-Processing)を参考により大きなデータセットを作成してみましょう。

また、今回はジョブを作成して s3 に出力しましたが、SageMaker Pipeline や Python スクリプトにエクスポートして定期実行することも可能です。

今回は s3 から csv ファイルをダウンロードしましたが、他にも Redshift や Athena などからクエリしてデータをインポートすることも可能です。